In [3]:
try:
    import cPickle as pickle
except:
    import pickle
# import plottools
import matplotlib
matplotlib.use('TkAgg')  # Specify the backend
import matplotlib.pyplot as plt
import matplotlib.text
import os
import numpy
import math
import collections
from collections import OrderedDict
import json
import pandas as pd
import matplotlib.gridspec as gridspec

In [4]:
def load_feature_data(path_model_results, path_exp_features, path_feature_categs, features_to_skip=[]):
    """Load model feature data and feature errors, load experimental features and the feature categories"""
    # Loading the model validation results: feature scores and errors
    data_dict = collections.OrderedDict()
    for models in os.listdir(path_model_results):
        data_dict[models] = {}
        with open(os.path.join(path_model_results, models, 'somatic_model_features.json')) as f:
            features = json.load(f, object_pairs_hook=collections.OrderedDict)
            features = OrderedDict((key, value) for key, value in features.items() if key not in features_to_skip)
            data_dict[models]['features'] = collections.OrderedDict(sorted(features.items()))
        with open(os.path.join(path_model_results, models, 'somatic_model_errors.json')) as f:
            errors = json.load(f, object_pairs_hook=collections.OrderedDict)
            errors = OrderedDict((key, value) for key, value in errors.items() if key not in features_to_skip)
            data_dict[models]['errors'] = collections.OrderedDict(sorted(errors.items()))
        with open(os.path.join(path_model_results, models, 'final_score.json')) as f:
            final_score = json.load(f)
            data_dict[models]['final score'] = final_score['score']            
    # Loading observation
    with open(path_exp_features) as f:
        observation = json.load(f, object_pairs_hook=collections.OrderedDict)
    # Loading feature categories
    with open(path_feature_categs) as f:
        all_features_dict = json.load(f, object_pairs_hook=collections.OrderedDict)
    feature_cat_names = []
    for key in all_features_dict:
        feature_cat_names.append(key)
    return data_dict, observation, all_features_dict, feature_cat_names

In [5]:
def create_blank_dataframe(feature_cats, models):
    print(feature_cats)
    print(models)
    dataframe = pd.DataFrame(index=models, columns=feature_cats + ["color", "NaN values"])
    dataframe["NaN values"] = dataframe["NaN values"].fillna(0)
    return dataframe

In [6]:
def add_morph_type(dataframe, models, point=[], stylized=[], detailed=[]):
    """Add the type of the model morphology to the dataframe"""
    for model in models:
        #print(model)
        if model in point:
            dataframe.loc[model, "morphology"] = "point"
        elif model in stylized:
            dataframe.loc[model, "morphology"] = "stylized"
        elif model in detailed:
            dataframe.loc[model, "morphology"] = "detailed"
        else:
            dataframe.loc[model, "morphology"] = None
            print("No information about the morphology type of the model", model)

In [7]:
def add_new_model_names(dataframe, new_names_dict):
    for model_name in new_names_dict:
        dataframe.loc[model_name, "new names"] = new_names_dict[model_name]

In [8]:
def add_model_colors(dataframe, models, path_color_file):
    with open(path_color_file, "r") as f:
        model_colors_dict = json.load(f)
    f.close()
    dataframe['color'] = dataframe['color'].astype('object')
    for model in models:
        try:
            dataframe.at[model, "color"] = model_colors_dict[dataframe.loc[model, "new names"]]
        except:
            print("no color info about", model)
            dataframe.at[model, "color"] = None

In [9]:
def plot_category_feat_error_means(data_dict, observation, all_features_dict, feature_cat, dataframe, zoomed=False, xlim_values=[]):
    """Calculate feature error means for the features in the category, plot and save the fig, also fills the dataframe"""
    features_in_cat_list = all_features_dict[feature_cat]
    # keep only those features that are present in observation
    features_in_cat_list = [feat for feat in features_in_cat_list if len(list(s for s in list(observation.keys())
                                                                               if feat in s)) != 0]
    plt.clf()
    plt.style.use('dark_background')
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    try:
        colormap = plt.cm.spectral  # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    except:
        colormap = plt.cm.nipy_spectral
    # plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, len(list(data_dict.keys()))))))
    plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, 21))))
    #plt.gcf().set_size_inches(210 / 25.4, (297 / 25.4) / 2)

    for key in list(data_dict.keys()):
        #print("model:",key)
        x = []
        all_nan_values = 0
        for feat in features_in_cat_list:
            #print("feat:", feat)
            feat_vals = list(v for k, v in data_dict[key]['errors'].items() if k.startswith(feat+"."))
            #x.append(numpy.mean(feat_vals))
            #print(feat_vals)
            nan_values = sum(math.isnan(x) for x in feat_vals)
            #print("nan_values:",nan_values)
            all_nan_values = all_nan_values + nan_values
            #print("all_nan_values:",all_nan_values)
            x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
        y = range(len(features_in_cat_list))
        if dataframe["morphology"][key] == "point":
            marker_type = 'o'
            marker_size = 7
        elif dataframe["morphology"][key] == "stylized":
            marker_type = '^'
            marker_size = 7
        elif dataframe["morphology"][key] == "detailed":
            marker_type = '*'
            marker_size = 10
        else:
            marker_type = ""
            marker_size = 0
        ax.plot(x, y, marker=marker_type, markersize=marker_size, linestyle='none',  clip_on=False, label=key)
        # print("x", x)
        dataframe.loc[ key, feature_cat] = numpy.nanmean(x)
        dataframe.loc[ key, "NaN values"] = dataframe.loc[ key, "NaN values"] + all_nan_values
        dataframe.loc[key, "final score"] = data_dict[key]["final score"]

    fig.set_figheight(len(features_in_cat_list) / 2)

    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=12, markerscale=1)

    ax.yaxis.set_ticks(range(len(features_in_cat_list)))
    ax.set_yticklabels(features_in_cat_list, fontsize=14)
    ax.set_ylim(-1, len(features_in_cat_list))
    ax.set_xlabel('Error Scores', fontsize=14)
    ax.set_ylabel('Features in the Category', fontsize=14)
    ax.set_title('The ' + feature_cat + ' category feature errors', fontsize=17, pad=10)
    plt.xticks(fontsize=14)
    plt.grid()
    plt.xlim(0)
    # plt.tick_params(labelsize=11)
    fig_name = feature_cat + '_feature_errors_means.png'
    plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')
    # Zoomed in figures
    if zoomed:
        xlim_value_1 = xlim_values[feature_cat_names.index(feat_cat)][0]
        xlim_value_2 = xlim_values[feature_cat_names.index(feat_cat)][1]
        if xlim_value_1 is not None and xlim_value_2 is not None:
            ax.set_xlim(xlim_value_1,xlim_value_2)
            fig_name = feature_cat + '_feature_errors_means_ZOOM.png'
            # plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')
    return dataframe

In [10]:
# !!! NOT IN USE - LATER CAN BE FINALIZED IF THERE IS TIME 
def fill_up_datatable_with_results(data_dict, observation, all_features_dict, feature_cat, dataframe):
    """Calculate feature error means for the features in the category, and fill the dataframe"""
    features_in_cat_list = all_features_dict[feature_cat]
    # keep only those features that are present in observation
    features_in_cat_list = [feat for feat in features_in_cat_list if len(list(s for s in list(observation.keys())
                                                                               if feat in s)) != 0]

    for key in list(data_dict.keys()):
        #print("model:",key)
        x = []
        all_nan_values = 0
        for feat in features_in_cat_list:
            #print("feat:", feat)
            feat_vals = list(v for k, v in data_dict[key]['errors'].items() if k.startswith(feat+"."))
            #x.append(numpy.mean(feat_vals))
            #print(feat_vals)
            nan_values = sum(math.isnan(x) for x in feat_vals)
            #print("nan_values:",nan_values)
            all_nan_values = all_nan_values + nan_values
            #print("all_nan_values:",all_nan_values)
            x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
        y = range(len(features_in_cat_list))
        ax.plot(x, y, marker=marker_type, markersize=marker_size, linestyle='none',  clip_on=False, label=key)
        # print("x", x)
        dataframe.loc[ key, feature_cat] = numpy.nanmean(x)
        dataframe.loc[ key, "NaN values"] = dataframe.loc[ key, "NaN values"] + all_nan_values
        dataframe.loc[key, "final score"] = data_dict[key]["final score"]

    return dataframe

In [11]:
def plot_all_cat_errors(dataframe, feature_cats, models, save_dir, zoomed=False, xlim_values=[]):
    """Plotting the category errors of the models"""

    plt.clf()
    plt.style.use('dark_background')
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    try:
        colormap = plt.cm.spectral  # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    except:
        colormap = plt.cm.nipy_spectral
    #plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, len(list(data_dict.keys()))))))
    plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, 21))))
    # plt.gcf().set_size_inches(230 / 25.4, (297 / 25.4) / 2)

    for model in models:
        if dataframe[model]["morphology"] == "point":
            marker_type = 'o'
            marker_size = 7
        elif dataframe[model]["morphology"] == "stylized":
            marker_type = '^'
            marker_size = 7
        elif dataframe[model]["morphology"] == "detailed":
            marker_type = '*'
            marker_size = 10
        else:
            marker_type = ""
            marker_size = 0
        ax.plot(dataframe[model][0:len(feature_cats)],feature_cats, marker=marker_type, markersize=marker_size, linestyle='none', label=model)

    fig.set_figheight(len(feature_cats))

    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=12, markerscale=1)

    ax.set_xlabel('Error Scores', fontsize=14)
    ax.set_ylabel('Feature Categories', fontsize=14)
    ax.set_title('Category Error Scores of the Models', fontsize=17, pad=10)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    # plt.xticks(rotation=45)
    plt.xlim(0)
    plt.grid()
    fig_name = 'ALL_Feature_Category_errors.png'
    plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')
    # plt.show()
    # Zoomed in figure
    if zoomed:
        xlim_value_1 = xlim_values[0]
        xlim_value_2 = xlim_values[1]
        if xlim_value_1 is not None and xlim_value_2 is not None:
            ax.set_xlim(xlim_value_1,xlim_value_2)
            fig_name = 'ALL_Feature_Category_errors_ZOOM.png'
            # plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')



In [12]:
def plot_nan_finalscore_white(dataframe, models, save_dir):
    
    plt.clf()
    fig = plt.figure(1, facecolor="white")
    plt.style.use('default')
    ax = fig.add_subplot(111)
    ax.set_facecolor("white")
    for model in models:
        if dataframe[model]["color"] is not None:
            if dataframe[model]["morphology"] == "point":
                marker_type = 'o'
                marker_size = 8
            elif dataframe[model]["morphology"] == "stylized":
                marker_type = '^'
                marker_size = 9
            elif dataframe[model]["morphology"] == "detailed":
                marker_type = '*'
                marker_size = 12
            else:
                marker_type = ""
                marker_size = 0
            # ax.plot(dataframe.loc["NaN values", model], float(dataframe.loc["final score", model]), marker=marker_type, markersize=marker_size, markeredgecolor="black", markeredgewidth=0.5, linestyle = 'None', label=dataframe.loc["new names", model], color = dataframe.loc["color", model])
            ax.plot(dataframe.loc["NaN values", model], float(dataframe.loc["final score", model]), marker=marker_type, markersize=marker_size, linestyle = 'None', label=dataframe.loc["new names", model], color = dataframe.loc["color", model])
            
    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=12, markerscale=1, frameon=False)
    fig.set_size_inches(12, 8)
    ax.set_xlabel('Count of Non-Evaluable Features', fontsize=14)
    ax.set_ylabel('Model Final Score', fontsize=14)
    ax.set_title('Finding the Best Models: Model Performance and Non-Evaluable Features', fontsize=17, pad=10)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    #plt.xlim(0)
    plt.grid()
    plt.xlim(0)
    plt.ylim(0)
    plt.tick_params(top=False, bottom=False, left=False, right=False)
    ax.spines['bottom'].set_color('grey')
    ax.spines['top'].set_color('grey')
    ax.spines['left'].set_color('grey')
    ax.spines['right'].set_color('grey')
    fig_name = 'Nonevaluable features_vs_finalscore_white_corrected_final.png'
    plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.ylim(0, 6)
    plt.xlim(0,15)
    ax.set_title('Number of non-evaluable features plotted against the model final score', fontsize=17, pad=10)
    fig_name = 'Nonevaluable features_vs_finalscore_ZOOM_white_corrected_final.png'
    #plt.savefig(os.path.join(save_dir, fig_name), bbox_extra_artists=(lgd,), bbox_inches='tight')
            

In [13]:
point_models = ["Neymotin_2011", "Neymotin_2013", "Stacey(rb)_2009", "Stacey(vd)_2009", "Vladimirov_2013", "Hu(rb)_2018", "Hummos_2014"]
stylized_models = ["Cuts&Poir_2015", "Lee_2014", "Turi_2019", "Saudargiene_2015", "Bezaire_2016"]
detailed_models = ["Saraga_2006_6.3_cels", "Tzilivaki1_2019_0.1_dt", "Tzilivaki4_2019_0.1_dt", "Ferr&Asco_2015", "Migliore_2018", "Tzilivaki3_2019_0.1_dt", "Tzilivaki2_2019_0.1_dt", "Tzilivaki5_2019_0.1_dt", "Chiovini_2014"]

new_names_dict = {
    "Neymotin_2011" : "Neymotin 2011",
    "Neymotin_2013" : "Neymotin 2013",
    "Stacey(rb)_2009" : "Stacey(rb) 2009", 
    "Stacey(vd)_2009" : "Stacey(vd) 2009", 
    "Vladimirov_2013" : "Vladimirov 2013", 
    "Hu(rb)_2018" : "Hu(rb) 2018", 
    #"Hu(vd)_2018" : "Hu(vd) 2018",
    #"Sherif_2020" : "Sherif 2020",
    "Hummos_2014" : "Hummos 2014", 
    "Cuts&Poir_2015" : "Cuts&Poir 2015",
    "Lee_2014" : "Lee 2014",
    "Turi_2019" : "Turi 2019",
    "Saudargiene_2015" : "Saudargiene 2015",
    "Bezaire_2016" : "Bezaire 2016",
    "Ferr&Asco_2015" : "Ferr&Asco 2015", 
    "Migliore_2018" : "Migliore 2018", 
    "Saraga_2006_6.3_cels" : "Saraga 2006 6.3C", 
    "Chiovini_2014" : "Chiovini 2014",
    "Tzilivaki1_2019" : "Tzilivaki.1 2019 no set dt", 
    "Tzilivaki2_2019" : "Tzilivaki.2 2019 no set dt", 
    "Tzilivaki3_2019" : "Tzilivaki.3 2019 no set dt", 
    "Tzilivaki4_2019" : "Tzilivaki.4 2019 no set dt",
    "Tzilivaki5_2019" : "Tzilivaki.5 2019 no set dt", 
    "Tzilivaki1_2019_0.1_dt" : "Tzilivaki.1 2019",
    "Tzilivaki2_2019_0.1_dt" : "Tzilivaki.2 2019",
    "Tzilivaki3_2019_0.1_dt" : "Tzilivaki.3 2019",
    "Tzilivaki4_2019_0.1_dt" : "Tzilivaki.4 2019",
    "Tzilivaki5_2019_0.1_dt" : "Tzilivaki.5 2019"
}

In [14]:
# path to model soma test validation results
path_model_results = os.path.abspath('/home/szabobogi/hippounit_standard_features/validation_results_MOUSE/results/somaticfeat/')
# path to experimental standard features fot validation 
path_exp_features = os.path.abspath('/home/szabobogi/BluePyEfe_standard_features/BluePyEfe/FINAL_PV_PTI_data/features_for_validation/standard_features_for_validation_MOUSE_FINAL.json')
# path to the somatic feature categories file 
path_feature_categs = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config//somatic_feature_chategories.json')
# path to model colors file
path_color_file = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config/modelcolors_vibrant_final.json')
# path to save directory
save_dir = "/home/szabobogi/hippounit_standard_features/visualization/figures_MOUSE/Finalscore_plot/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [15]:
# exclude these features from the data_dict as they not really work or make no sense
# load the data
data_dict, observation, all_features_dict, feature_cat_names = load_feature_data(path_model_results, path_exp_features, path_feature_categs)

# create blank dataframe
model_names = list(data_dict.keys())
df_cat_errors = create_blank_dataframe(feature_cat_names, model_names)

add_morph_type(df_cat_errors, model_names, point=point_models, stylized = stylized_models, detailed=detailed_models)

# Adding new more sophisticated names to the models in a new colunm
add_new_model_names(df_cat_errors, new_names_dict)
# Sorting the models in alphabetical order in the table
df_cat_errors.sort_index(key=lambda x: x.str.lower(), inplace = True)

add_model_colors(df_cat_errors, model_names, path_color_file)

# plot the feature errors in the feature categories and fill the dataframe with the category errors
# set xlim_vales for the figures if needed:
zoomed = False
xlim_values = [[0,10], [0,7.5], [0, 3], [0,5], [0, 11], [None, None]]
for feat_cat in feature_cat_names:
    df_cat_errors = plot_category_feat_error_means(data_dict, observation, all_features_dict, feat_cat, df_cat_errors, zoomed, xlim_values)
matplotlib.pyplot.close()

# plot the category errors of the models
# set xlim_vales for the figures if needed:
zoomed = False
xlim_values = [0, 13]
plot_all_cat_errors(df_cat_errors.T, feature_cat_names, model_names, save_dir, zoomed, xlim_values)

# to have the models in the right order 
df_point = df_cat_errors[df_cat_errors["morphology"] == "point"]
df_stylized = df_cat_errors[df_cat_errors["morphology"] == "stylized"]
df_detailed = df_cat_errors[df_cat_errors["morphology"] == "detailed"]
df_cat_errors = pd.concat([df_point, df_stylized, df_detailed])

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
plt.close()
plt.style.use('default')
plot_nan_finalscore_white(df_cat_errors.T, df_cat_errors.index, save_dir)

df_cat_errors_1 = df_cat_errors
display(df_cat_errors_1)

['AP_shape', 'AHP', 'adaptation', 'pattern', 'subthreshold']
['Tzilivaki1_2019_0.1_dt', 'Lee_2014', 'Tzilivaki5_2019', 'Cuts&Poir_2015', 'Tzilivaki2_2019', 'Tzilivaki2_2019_0.1_dt', 'Turi_2019', 'Bezaire_2016', 'Tzilivaki4_2019_0.1_dt', 'Vladimirov_2013', 'Stacey(vd)_2009', 'Tzilivaki4_2019', 'Sherif_2020', 'Hu(rb)_2018', 'Saraga_2006_6.3_cels', 'Tzilivaki3_2019_0.1_dt', 'Saudargiene_2015', 'Chiovini_2014', 'Migliore_2018', 'Hummos_2014', 'Neymotin_2013', 'Neymotin_2011', 'Hu(vd)_2018', 'Stacey(rb)_2009', 'Tzilivaki5_2019_0.1_dt', 'Ferr&Asco_2015', 'Tzilivaki3_2019']
No information about the morphology type of the model Tzilivaki5_2019
No information about the morphology type of the model Tzilivaki2_2019
No information about the morphology type of the model Tzilivaki4_2019
No information about the morphology type of the model Sherif_2020
No information about the morphology type of the model Hu(vd)_2018
No information about the morphology type of the model Tzilivaki3_2019
no color info 

/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???


,AP_shape,AHP,adaptation,pattern,subthreshold,color,NaN values,morphology,new names,final score
Hu(rb)_2018,1.42047,3.873849,5.882274,3.047755,5.23881,"[1.0, 0.6485300602947661, 0.0, 1.0]",0.0,point,Hu(rb) 2018,4.021963826634831
Hummos_2014,3.298316,4.078806,3.525153,2.049983,2.680394,"[0.7724252430134786, 1.0, 0.0, 1.0]",0.0,point,Hummos 2014,3.0347530013809574
Neymotin_2011,1.670959,1.944368,1.698162,2.241506,1.355847,"[0.0, 0.7, 0.4, 1.0]",0.0,point,Neymotin 2011,1.63752947221106
Neymotin_2013,1.63893,1.810066,1.609846,1.960389,1.359561,"[0.0, 0.9889734834587774, 1.0, 1.0]",0.0,point,Neymotin 2013,1.6910901551858202
Stacey(rb)_2009,1.516282,2.036691,1.581986,9.149479,1.947407,"[0.0, 0.38676699889935195, 1.0, 1.0]",0.0,point,Stacey(rb) 2009,3.2157076974556813
Stacey(vd)_2009,1.669993,1.949527,1.573816,1.951827,1.387132,"[0.7944841823518297, 0.0, 1.0, 1.0]",0.0,point,Stacey(vd) 2009,1.690438716257492
Vladimirov_2013,2.147804,2.710978,1.731272,1.512224,9.870368,"[1.0, 0.0, 0.6264711209564149, 1.0]",0.0,point,Vladimirov 2013,2.6638327870501795
Bezaire_2016,4.326752,6.075161,1.56274,3.989632,1.148263,"[1.0, 0.532721120956415, 0.0, 1.0]",0.0,stylized,Bezaire 2016,3.3088179324918983
Cuts&Poir_2015,3.311952,4.688853,1.635786,6.233609,1.942754,"[0.14705697058638245, 1.0, 0.0, 1.0]",0.0,stylized,Cuts&Poir 2015,3.6412907199939264
Lee_2014,2.6171,3.996983,4.91812,1.790088,1.944922,"[0.0, 0.8036791805174157, 1.0, 1.0]",0.0,stylized,Lee 2014,2.7540173041323643


In [16]:
# path to model soma test validation results
path_model_results = os.path.abspath('/home/szabobogi/hippounit_standard_features/validation_results_RAT/results/somaticfeat/')
# path to experimental standard features fot validation 
path_exp_features = os.path.abspath('/home/szabobogi/BluePyEfe_standard_features/BluePyEfe/FINAL_PV_PTI_data/features_for_validation/standard_features_for_validation_RAT_FINAL.json')
# path to the somatic feature categories file 
path_feature_categs = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config//somatic_feature_chategories.json')
# path to model colors file
path_color_file = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config/modelcolors_vibrant_final.json')
# path to save directory
save_dir = "/home/szabobogi/hippounit_standard_features/visualization/figures_RAT/Finalscore_plot/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [17]:
# exclude these features from the data_dict as they not really work or make no sense
# load the data
data_dict, observation, all_features_dict, feature_cat_names = load_feature_data(path_model_results, path_exp_features, path_feature_categs)

# create blank dataframe
model_names = list(data_dict.keys())
df_cat_errors = create_blank_dataframe(feature_cat_names, model_names)

add_morph_type(df_cat_errors, model_names, point=point_models, stylized = stylized_models, detailed=detailed_models)

# Adding new more sophisticated names to the models in a new colunm
add_new_model_names(df_cat_errors, new_names_dict)
# Sorting the models in alphabetical order in the table
df_cat_errors.sort_index(key=lambda x: x.str.lower(), inplace = True)

add_model_colors(df_cat_errors, model_names, path_color_file)

# plot the feature errors in the feature categories and fill the dataframe with the category errors
# set xlim_vales for the figures if needed:
zoomed = False
xlim_values = [[0,10], [0,7.5], [0, 3], [0,5], [0, 11], [None, None]]
for feat_cat in feature_cat_names:
    df_cat_errors = plot_category_feat_error_means(data_dict, observation, all_features_dict, feat_cat, df_cat_errors, zoomed, xlim_values)
matplotlib.pyplot.close()

# plot the category errors of the models
# set xlim_vales for the figures if needed:
zoomed = False
xlim_values = [0, 13]
plot_all_cat_errors(df_cat_errors.T, feature_cat_names, model_names, save_dir, zoomed, xlim_values)

# to have the models in the right order 
df_point = df_cat_errors[df_cat_errors["morphology"] == "point"]
df_stylized = df_cat_errors[df_cat_errors["morphology"] == "stylized"]
df_detailed = df_cat_errors[df_cat_errors["morphology"] == "detailed"]
df_cat_errors = pd.concat([df_point, df_stylized, df_detailed])

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
plt.close()
plt.style.use('default')
plot_nan_finalscore_white(df_cat_errors.T, df_cat_errors.index, save_dir)


df_cat_errors_2 = df_cat_errors
display(df_cat_errors_2)

['AP_shape', 'AHP', 'adaptation', 'pattern', 'subthreshold']
['Tzilivaki1_2019_0.1_dt', 'Lee_2014', 'Tzilivaki5_2019', 'Cuts&Poir_2015', 'Tzilivaki2_2019', 'Tzilivaki2_2019_0.1_dt', 'Turi_2019', 'Bezaire_2016', 'Tzilivaki4_2019_0.1_dt', 'Vladimirov_2013', 'Stacey(vd)_2009', 'Tzilivaki4_2019', 'Sherif_2020', 'Hu(rb)_2018', 'Saraga_2006_6.3_cels', 'Tzilivaki3_2019_0.1_dt', 'Saudargiene_2015', 'Chiovini_2014', 'Migliore_2018', 'Hummos_2014', 'Neymotin_2013', 'Neymotin_2011', 'Hu(vd)_2018', 'Stacey(rb)_2009', 'Tzilivaki5_2019_0.1_dt', 'Ferr&Asco_2015', 'CA1_Bezaire_2016_(Lee)_standard', 'Tzilivaki3_2019']
No information about the morphology type of the model Tzilivaki5_2019
No information about the morphology type of the model Tzilivaki2_2019
No information about the morphology type of the model Tzilivaki4_2019
No information about the morphology type of the model Sherif_2020
No information about the morphology type of the model Hu(vd)_2018
No information about the morphology type of the m

/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
/tmp/ipykernel_2124219/3020567059.py:32: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???


,AP_shape,AHP,adaptation,pattern,subthreshold,color,NaN values,morphology,new names,final score
Hu(rb)_2018,1.458814,3.268277,6.110559,4.701079,5.150026,"[1.0, 0.6485300602947661, 0.0, 1.0]",0.0,point,Hu(rb) 2018,4.207940754855265
Hummos_2014,3.424731,3.665935,3.874315,2.720322,1.356414,"[0.7724252430134786, 1.0, 0.0, 1.0]",0.0,point,Hummos 2014,2.8526428658353113
Neymotin_2011,1.898667,1.723444,1.530988,2.370891,2.158131,"[0.0, 0.7, 0.4, 1.0]",0.0,point,Neymotin 2011,2.010604375491086
Neymotin_2013,1.91017,1.810174,1.45262,2.808193,1.402459,"[0.0, 0.9889734834587774, 1.0, 1.0]",0.0,point,Neymotin 2013,1.9702758371161502
Stacey(rb)_2009,1.90564,1.852202,1.808275,8.25367,2.05511,"[0.0, 0.38676699889935195, 1.0, 1.0]",0.0,point,Stacey(rb) 2009,3.2645044499823417
Stacey(vd)_2009,1.907215,1.78845,1.486476,2.445583,1.369078,"[0.7944841823518297, 0.0, 1.0, 1.0]",0.0,point,Stacey(vd) 2009,1.8511437437699414
Vladimirov_2013,1.599277,2.563669,1.726826,3.343096,10.229984,"[1.0, 0.0, 0.6264711209564149, 1.0]",0.0,point,Vladimirov 2013,2.9916211224607885
Bezaire_2016,3.901937,5.492973,1.91984,3.512671,0.788531,"[1.0, 0.532721120956415, 0.0, 1.0]",0.0,stylized,Bezaire 2016,2.9389301378106785
Cuts&Poir_2015,2.747609,4.364414,1.956284,6.057391,1.308421,"[0.14705697058638245, 1.0, 0.0, 1.0]",0.0,stylized,Cuts&Poir 2015,3.2313778497447854
Lee_2014,2.438696,3.72851,6.363629,3.795993,1.622101,"[0.0, 0.8036791805174157, 1.0, 1.0]",0.0,stylized,Lee 2014,3.160513410513901


In [18]:
df_cat_errors_1 = df_cat_errors_1.rename(columns={'final score': 'final score mouse', 'color': 'color', 'morphology': 'morphology', 'new names': 'new names'})
df_cat_errors_2 = df_cat_errors_2.rename(columns={'final score': 'final score rat'})

# Create the new dataframe with only the 'final score mouse' and 'final score rat' columns
df_combined = pd.DataFrame(index=df_cat_errors_1.index)
df_combined['final score mouse'] = df_cat_errors_1['final score mouse']
df_combined['final score rat'] = df_cat_errors_2['final score rat']
df_combined['color'] = df_cat_errors_1['color']
df_combined['morphology'] = df_cat_errors_1['morphology']
df_combined['new names'] = df_cat_errors_1['new names']

# Save the new dataframe to a CSV file
df_combined.to_csv('combined_final_scores.csv')

In [19]:
display(df_combined)

,final score mouse,final score rat,color,morphology,new names
Hu(rb)_2018,4.021963826634831,4.207940754855265,"[1.0, 0.6485300602947661, 0.0, 1.0]",point,Hu(rb) 2018
Hummos_2014,3.0347530013809574,2.8526428658353113,"[0.7724252430134786, 1.0, 0.0, 1.0]",point,Hummos 2014
Neymotin_2011,1.63752947221106,2.010604375491086,"[0.0, 0.7, 0.4, 1.0]",point,Neymotin 2011
Neymotin_2013,1.6910901551858202,1.9702758371161502,"[0.0, 0.9889734834587774, 1.0, 1.0]",point,Neymotin 2013
Stacey(rb)_2009,3.2157076974556813,3.2645044499823417,"[0.0, 0.38676699889935195, 1.0, 1.0]",point,Stacey(rb) 2009
Stacey(vd)_2009,1.690438716257492,1.8511437437699414,"[0.7944841823518297, 0.0, 1.0, 1.0]",point,Stacey(vd) 2009
Vladimirov_2013,2.6638327870501795,2.9916211224607885,"[1.0, 0.0, 0.6264711209564149, 1.0]",point,Vladimirov 2013
Bezaire_2016,3.3088179324918983,2.9389301378106785,"[1.0, 0.532721120956415, 0.0, 1.0]",stylized,Bezaire 2016
Cuts&Poir_2015,3.6412907199939264,3.2313778497447854,"[0.14705697058638245, 1.0, 0.0, 1.0]",stylized,Cuts&Poir 2015
Lee_2014,2.7540173041323643,3.160513410513901,"[0.0, 0.8036791805174157, 1.0, 1.0]",stylized,Lee 2014


In [22]:
def plot_finalscores(dataframe, models, save_dir):

    plt.clf()
    fig = plt.figure(1, facecolor="white")
    plt.style.use('default')
    ax = fig.add_subplot(111)
    ax.set_facecolor("white")
    for model in models:
        if dataframe[model]["color"] is not None:
            if dataframe[model]["morphology"] == "point":
                marker_type = 'o'
                marker_size = 15
            elif dataframe[model]["morphology"] == "stylized":
                marker_type = '^'
                marker_size = 15
            elif dataframe[model]["morphology"] == "detailed":
                marker_type = '*'
                marker_size = 22
            else:
                marker_type = ""
                marker_size = 0
            ax.plot(float(dataframe.loc["final score mouse", model]), float(dataframe.loc["final score rat", model]), 
                    marker=marker_type, markersize=marker_size, linestyle = 'None', label=dataframe.loc["new names", model], 
                    color = dataframe.loc["color", model], alpha=0.75)
            
    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=17, markerscale=0.8, frameon=False)
    fig.set_size_inches(10, 10)
    ax.set_xlabel('Mouse Final Score', fontsize=22, labelpad=10)
    ax.set_ylabel('Rat Final Score', fontsize=22, labelpad=10)
    
    # ax.set_title('Finding the Best Models: Model Performance and Non-Evaluable Features', fontsize=17, pad=10)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    #plt.xlim(0)
    # plt.grid()
    plt.xlim(0)
    plt.ylim(0)

    # Get the limits of the plot (x and y axis limits)
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    # Define the limits for the diagonal line (min and max for both axes)
    line_min = min(x_min, y_min)
    line_max = max(x_max, y_max)
    # Plot the light grey diagonal line (y=x) using the calculated limits
    ax.plot([line_min, line_max], [line_min, line_max], color='lightgrey', linestyle='--', zorder=0)
    
    # Adjust axis limits to accommodate the diagonal line
    ax.set_xlim([line_min, line_max])
    ax.set_ylim([line_min, line_max])

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    plt.tick_params(top=False, bottom=False, left=False, right=False)
    #ax.spines['bottom'].set_color('grey')
    #ax.spines['top'].set_color('grey')
    #ax.spines['left'].set_color('grey')
    #ax.spines['right'].set_color('grey')
    fig_name = 'MOUSE_vs_RAT_small'
    save_plot_as_svg = os.path.join(save_dir, f"{fig_name}.svg")
    plt.savefig(save_plot_as_svg, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600, format='svg')
    plt.show()


In [23]:
plot_finalscores(df_combined.T, df_combined.index, save_dir)